# --- `Day 4`: Passport Processing ---

In [1]:
import aocd
import re
import operator
from itertools import combinations
from functools import reduce

def prod(iterable):
    return reduce(operator.mul, iterable, 1)

def count(iterable, predicate = bool):
    return sum([1 for item in iterable if predicate(item)])

In [29]:
def parse_line(section): 
    return dict(re.findall(r'([a-z]+):([^\s]+)', section))
    
def parse_input(input):
    return list(map(parse_line, input.split("\n\n")))

In [27]:
final_input = parse_input(aocd.get_data(day=4, year=2020))
final_input[:2]

[{'eyr': '2029',
  'iyr': '2013',
  'hcl': '#ceb3a1',
  'byr': '1939',
  'ecl': 'blu',
  'hgt': '163cm',
  'pid': '660456119'},
 {'hcl': '#0f8b2e',
  'ecl': 'grn',
  'byr': '1975',
  'iyr': '2011',
  'eyr': '2028',
  'cid': '207',
  'hgt': '158cm',
  'pid': '755567813'}]

In [28]:
test_input = parse_input('''\
ecl:gry pid:860033327 eyr:2020 hcl:#fffffd
byr:1937 iyr:2017 cid:147 hgt:183cm

iyr:2013 ecl:amb cid:350 eyr:2023 pid:028048884
hcl:#cfa07d byr:1929

hcl:#ae17e1 iyr:2013
eyr:2024
ecl:brn pid:760753108 byr:1931
hgt:179cm

hcl:#cfa07d eyr:2025 pid:166559648
iyr:2011 ecl:brn hgt:59in
''')

test_input

[{'ecl': 'gry',
  'pid': '860033327',
  'eyr': '2020',
  'hcl': '#fffffd',
  'byr': '1937',
  'iyr': '2017',
  'cid': '147',
  'hgt': '183cm'},
 {'iyr': '2013',
  'ecl': 'amb',
  'cid': '350',
  'eyr': '2023',
  'pid': '028048884',
  'hcl': '#cfa07d',
  'byr': '1929'},
 {'hcl': '#ae17e1',
  'iyr': '2013',
  'eyr': '2024',
  'ecl': 'brn',
  'pid': '760753108',
  'byr': '1931',
  'hgt': '179cm'},
 {'hcl': '#cfa07d',
  'eyr': '2025',
  'pid': '166559648',
  'iyr': '2011',
  'ecl': 'brn',
  'hgt': '59in'}]

## Solution 1

In [63]:
required_fields = {'hcl', 'iyr', 'eyr', 'ecl', 'pid', 'byr', 'hgt'}
valid_passport = required_fields.issubset

print(valid_passport({
  'ecl': 'gry',
  'pid': '860033327',
  'eyr': '2020',
  'hcl': '#fffffd',
  'byr': '1937',
  'iyr': '2017',
  'cid': '147',
  'hgt': '183cm'}))
print(valid_passport({
  'iyr': '2013',
  'ecl': 'amb',
  'cid': '350',
  'eyr': '2023',
  'pid': '028048884',
  'hcl': '#cfa07d',
  'byr': '1929'}))

True
False


In [43]:
def solve_1(input):
    return count(input, valid_passport)

assert solve_1(test_input) == 2

In [44]:
f"Solution 1: {solve_1(final_input)}"

'Solution 1: 204'

## Solution 2

- byr (Birth Year) - four digits; at least 1920 and at most 2002.
- iyr (Issue Year) - four digits; at least 2010 and at most 2020.
- eyr (Expiration Year) - four digits; at least 2020 and at most 2030.
- hgt (Height) - a number followed by either cm or in:
    - If cm, the number must be at least 150 and at most 193.
    - If in, the number must be at least 59 and at most 76.
- hcl (Hair Color) - a # followed by exactly six characters 0-9 or a-f.
- ecl (Eye Color) - exactly one of: amb blu brn gry grn hzl oth.
- pid (Passport ID) - a nine-digit number, including leading zeroes.
- cid (Country ID) - ignored, missing or not.

In [111]:
field_validators = dict(
    iyr = lambda v: 2010 <= int(v) <= 2020,
    eyr = lambda v: 2020 <= int(v) <= 2030,
    hcl = lambda v: re.match(r'#[0-9a-f]{6}$', v),
    ecl = lambda v: re.match(r'(amb|blu|brn|gry|grn|hzl|oth)', v),
    byr = lambda v: 1920 <= int(v) <= 2002,
    hgt = lambda v: ((v.endswith('cm') and (150 <= int(v[:-2]) <= 193)) or
        (v.endswith('in') and (59 <= int(v[:-2]) <= 76))),
    pid = lambda v: re.match(r'[0-9]{9}$', v)
)
assert field_validators['iyr']("2000") == False
assert field_validators['iyr']("2010")
assert field_validators['pid']("012345678")
assert field_validators['pid']("a012345678") == None
assert field_validators['pid']("0123456789") == None
assert field_validators['ecl']("brn")
assert field_validators['ecl']("ylw") == None
assert field_validators['hcl']("#1234ab")
assert field_validators['hcl']("#123456ab") == None
assert field_validators['hgt']("155cm")

def valid_passport_2(passport):
    return (valid_passport(passport) and 
        all(field_validators[field](passport[field]) 
            for field in required_fields))

In [112]:
def solve_2(input):
    return count(input, valid_passport_2)
    
assert solve_2(test_input) == 2

In [113]:
f"Solution 2: {solve_2(final_input)}"

'Solution 2: 179'